# Case Study: The Beazley Case

In [1]:
import sql

In [2]:
%load_ext sql
%sql sqlite:///tx_deathrow_small.db
%config SqlMagic.autocommit=False

**Please write your queries below in the empty boxes same as you write in a SQL Window**

Just append **%sql** before your query like this '%sql Select * from executions limit 5;'

In [4]:
%sql Select * from previous limit 2

 * sqlite:///tx_deathrow_small.db
(sqlite3.OperationalError) no such table: previous
[SQL: Select * from previous limit 2]
(Background on this error at: http://sqlalche.me/e/e3q8)


**About the Dataset and Schema**

In 1994, Napoleon Beazley shot 63-year-old businessman John Luttig in his garage while trying to steal his family’s car. Because he was just shy of 18 years old at the time of the murder, the Beazley case ignited a fierce debate over the death penalty for juvenile offenders. 3 years after Beazley was executed, the Supreme Court prohibited the execution of offenders under 18 at the time of their offense (Roper v Simmons, 2005).

The case was also notable because the victim was the father of a federal judge John Michael Luttig. During the appeals to the Supreme court, three of the nine justices recused themselves because of their personal ties to Judge Luttig, leaving only six to review the case.

Napoleon Beazley made an impassioned last statement arguing that an eye for an eye does not constitute justice. Our task is to retrieve his statement from the database.

The database has one table, executions. To begin with, we'll look at the table. It contains the following columns:

*	Execution
*	Date of Birth
*	Date of Offence
*	Highest Education Level
*	Last Name
*	First Name
*	TDCJ Number
*	Age at Execution
*	Date Received
*	Execution Date
*	Race
*	County
*	Eye Color
*	Weight
*	Height
*	Native County
*	Native State
*	Last Statement




## Recalling Filtering

The WHERE Block
The WHERE block allows us to filter the table for rows that meet certain conditions. Its format is WHERE <clause> and it always goes after the FROM block. Here, a clause refers to a Boolean statement that the computer can evaluate to be true or false like ex_number = 145. You can imagine that the computer will go through each row in the table checking if the clause is true, and if so, return the row

**Find the first and last names and ages (ex_age) of inmates 25 or younger at time of execution? How many are they?**

In [3]:
%sql SELECT first_name, last_name, ex_age FROM executions \
     WHERE ex_age <= 25; 

 * sqlite:///tx_deathrow_small.db
Done.


first_name,last_name,ex_age
Toronto,Patterson,24
T.J.,Jones,25
Napoleon,Beazley,25
Richard,Andrade,25
Jay,Pinkerton,24
Jesse,De La Rosa,24


In [4]:
%sql SELECT count(*) from executions\
     WHERE ex_age <= 25; 

 * sqlite:///tx_deathrow_small.db
Done.


count(*)
6


**Find the result if some named like Raymond(First Name) Landry(Last Name) was executed? What was his age?**

In [5]:
%sql SELECT ex_age FROM executions \
     WHERE first_name = 'Raymond' AND last_name = 'Landry';

 * sqlite:///tx_deathrow_small.db
Done.


ex_age


**Find Napoleon Beazley's last statement. What did he say?**

In [6]:
%sql SELECT last_statement FROM executions \
     WHERE first_name = 'Napoleon' AND last_name = 'Beazley';

 * sqlite:///tx_deathrow_small.db
Done.


last_statement
"The act I committed to put me here was not just heinous, it was senseless. But the person that committed that act is no longer here - I am. I'm not going to struggle physically against any restraints. I'm not going to shout, use profanity or make idle threats. Understand though that I'm not only upset, but I'm saddened by what is happening here tonight. I'm not only saddened, but disappointed that a system that is supposed to protect and uphold what is just and right can be so much like me when I made the same shameful mistake. If someone tried to dispose of everyone here for participating in this killing, I'd scream a resounding, ""No."" I'd tell them to give them all the gift that they would not give me...and that's to give them all a second chance. I'm sorry that I am here. I'm sorry that you're all here. I'm sorry that John Luttig died. And I'm sorry that it was something in me that caused all of this to happen to begin with. Tonight we tell the world that there are no second chances in the eyes of justice...Tonight, we tell our children that in some instances, in some cases, killing is right. This conflict hurts us all, there are no SIDES. The people who support this proceeding think this is justice. The people that think that I should live think that is justice. As difficult as it may seem, this is a clash of ideals, with both parties committed to what they feel is right. But who's wrong if in the end we're all victims? In my heart, I have to believe that there is a peaceful compromise to our ideals. I don't mind if there are none for me, as long as there are for those who are yet to come. There are a lot of men like me on death row - good men - who fell to the same misguided emotions, but may not have recovered as I have. Give those men a chance to do what's right. Give them a chance to undo their wrongs. A lot of them want to fix the mess they started, but don't know how. The problem is not in that people aren't willing to help them find out, but in the system telling them it won't matter anyway. No one wins tonight. No one gets closure. No one walks away victorious."


## Possible Innocence

Opponents of the death penalty have argued that the risk of mistakenly executing an innocent person is too great a cost to bear. In this chapter, we attempt to approximate how many innocent people may have been executed.

**Aggregate Functions**

There are two numbers we need to calculate the proportion:

  Numerator: Number of executions with claims of innocence.

  Denominator: Number of executions in total.

Until now, each row in the output has come from a single row of input. However, here we have both the numerator and denominator requiring information from multiple rows of input. This tells us we need to use an aggregate function. To “aggregate” means to combine multiple elements into a whole. Similarly, aggregation functions take multiple rows of data and combine them into one number.

    

**Find how many inmates provided last statements? And How many declined? Hint: Null Last Statement**

In [7]:
%sql SELECT count(*) from executions \
     WHERE last_statement IS NOT NULL;

 * sqlite:///tx_deathrow_small.db
Done.


count(*)
443


In [8]:
%sql SELECT count(*) FROM executions \
     WHERE last_statement IS NULL;

 * sqlite:///tx_deathrow_small.db
Done.


count(*)
110


**Find the total number of executions where inmates were over the age of 50?**

In [9]:
%sql SELECT count(*) FROM executions \
     WHERE ex_age > 50;

 * sqlite:///tx_deathrow_small.db
Done.


count(*)
68


**Find the average length (based on character count) of last statements in the dataset?**

In [10]:
%sql SELECT avg(length(last_statement)) FROM executions; 

 * sqlite:///tx_deathrow_small.db
Done.


avg(length(last_statement))
537.492099322799


**Find the proportion of inmates with claims of innocence in their last statements? Hint use Like %innocent%**

In [11]:
%sql SELECT count(*) FROM executions \
     WHERE last_statement = '%innocent%';

 * sqlite:///tx_deathrow_small.db
Done.


count(*)
0


**Long Tails**

Long tails refer to small numbers of samples which occur a large number of times. When we plot these out, they form a small sliver far to the right of the center of mass which looks like a tail. Long tails indicate the presence of outliers whose unusual behaviors may be of interest to us.

<img src="https://selectstarsql.com/imgs/execution_tail.png" width="500" height="500">

In context of Texas executions, the long tail refers to a small number of counties which have been known to conduct a large number of executions. Let’s find the percentage of executions from each county so that we can pick out the ones in the tail.Let’s find the percentage of executions from each county so that we can pick out the ones in the tail.

The Beazley chapter dealt with individual rows of data, but it’s clear that we need to do some aggregation to find county-level data. The Claims of Innocence chapter taught us aggregation, but those functions would end up aggregating the dataset into one row when we really want one row per county.

**The GROUP BY Block**

This is where the GROUP BY block comes in. It allows us to split up the dataset and apply aggregate functions within each group, resulting in one row per group. Its most basic form is GROUP BY <column>, <column>, ... and comes after the WHERE block.

**Find number of Executions by each county sorted by the highest? Name the count as “county_executions”**

In [13]:
%sql SELECT county,count(*) as county_executions FROM executions \
     GROUP BY county \
     ORDER BY county_executions DESC; 

 * sqlite:///tx_deathrow_small.db
Done.


county,county_executions
Harris,128
Dallas,58
Bexar,46
Tarrant,41
Nueces,16
Montgomery,15
Jefferson,15
Lubbock,13
Smith,12
Brazos,12


**Modify this query to find the number of executions from each county with and without a last statement? Hint: Rowwise Last Execution Statement**

In [14]:
%sql SELECT county,count(*) as county_executions FROM executions \
     WHERE last_statement IS NULL \
     GROUP BY county \
     ORDER BY county_executions DESC; 

 * sqlite:///tx_deathrow_small.db
Done.


county,county_executions
Harris,33
Bexar,9
Tarrant,6
Dallas,6
Nueces,5
Jefferson,4
Potter,3
Montgomery,3
Galveston,3
Wilbarger,2


In [15]:
%sql SELECT county,count(*) as county_executions FROM executions \
     WHERE last_statement IS NOT null \
     GROUP BY county \
     ORDER BY county_executions DESC; 

 * sqlite:///tx_deathrow_small.db
Done.


county,county_executions
Harris,95
Dallas,52
Bexar,37
Tarrant,35
Smith,12
Montgomery,12
Lubbock,12
Nueces,11
Jefferson,11
Brazos,10


**Perform Above Query with a CASE WHEN Solution in Columnar Approach**

In [16]:
%sql SELECT county, count(CASE WHEN last_statement is null THEN 'null' ELSE last_statement END) as county_executions FROM executions \
     GROUP BY county \
     ORDER BY county_executions DESC;

 * sqlite:///tx_deathrow_small.db
Done.


county,county_executions
Harris,128
Dallas,58
Bexar,46
Tarrant,41
Nueces,16
Montgomery,15
Jefferson,15
Lubbock,13
Smith,12
Brazos,12


**Count the number of inmates aged 50 or older that were executed in each county? Use “where” method.**

In [23]:
%sql SELECT county, count(*) FROM executions \
     WHERE ex_age >= 50 \
     GROUP BY county \
     ORDER BY count(*) DESC;

 * sqlite:///tx_deathrow_small.db
Done.


county,count(*)
Harris,21
Dallas,11
Montgomery,5
Tarrant,4
Lubbock,3
Smith,2
Lamar,2
Jefferson,2
Gregg,2
Galveston,2


**List the counties in which more than 2 inmates aged 50 or older have been executed. Hint Use Having**

In [5]:
%sql SELECT county, count(*) FROM executions \
     WHERE ex_age >= 50 \
     GROUP BY county \
     HAVING count(*) > 2;

 * sqlite:///tx_deathrow_small.db
Done.


county,count(*)
Dallas,11
Harris,21
Lubbock,3
Montgomery,5
Tarrant,4


**Find the first and last name of the the inmate with the longest last statement (by character count).**

In [29]:
%sql SELECT first_name, last_name, count(length(last_statement)) FROM executions; 

 * sqlite:///tx_deathrow_small.db
Done.


first_name,last_name,count(length(last_statement))
Christopher Anthony,Young,443


**Find the percentage of executions from each county.**

In [30]:
%sql SELECT county, (Count(county)* 100 / (SELECT Count(*) FROM executions)) as Percentage_execution \
From executions \
GROUP BY county \
ORDER BY Percentage_execution DESC;

 * sqlite:///tx_deathrow_small.db
Done.


county,Percentage_execution
Harris,23
Dallas,10
Bexar,8
Tarrant,7
Smith,2
Nueces,2
Montgomery,2
Lubbock,2
Jefferson,2
Brazos,2


**Harris County**

Is it surprising that Harris (home to the city of Houston), Dallas, Bexar and Tarrant account for about 50% of all executions in Texas